In [1]:
from openai import OpenAI
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.beta.assistant_stream_event import ThreadMessageDelta
from openai.types.beta.threads.text_delta_block import TextDeltaBlock 
import os
import pandas as pd

In [2]:
df_scrape = pd.read_pickle('/app/Store/dataframe.pkl')
df_asst = pd.read_csv('/app/Store/assistant_files.csv')
df_asst.rename(columns={'filename': 'file_path'}, inplace=True)

# Merging on the common column 'key'
df_merge = pd.merge(df_scrape, df_asst, on='file_path', how='inner')



In [3]:
result = df_merge[df_merge["file_id"]=='file-DmxSHWuwRB5jv7aWD32V7b']
citation_url = result['url'][0]

In [4]:
load_dotenv('/app/.env')

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]


def get_asst_by_name(client,name):
    l = client.beta.assistants.list()
    for asst in l:
        asst_dict = asst.to_dict()
        if asst_dict["name"] == name:
            return asst_dict["id"]
        
    return None
        
# Initialise the OpenAI client, and retrieve the assistant
client = OpenAI(api_key=OPENAI_API_KEY)
assistant = client.beta.assistants.retrieve(assistant_id="asst_WvxFY77dyhKaV6ei809wTfye")

In [5]:
user_query = 'What are some ways that I can learn about meditation?'



run = client.beta.threads.create_and_run_poll(
  assistant_id="asst_WvxFY77dyhKaV6ei809wTfye",
  thread={
    "messages": [
      {"role": "user", "content": user_query}
    ]
  },
  tool_choice="required"
)





In [6]:
thread_messages = client.beta.threads.messages.list(run.thread_id)

In [7]:
print(thread_messages)

SyncCursorPage[Message](data=[Message(id='msg_pS1iI3AKSbimg7N5tzeqYHh3', assistant_id='asst_WvxFY77dyhKaV6ei809wTfye', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=559, file_citation=FileCitation(file_id='file-F8ec6VTMZo9WxFuSj5LUh4'), start_index=547, text='【4:2†source】', type='file_citation'), FileCitationAnnotation(end_index=886, file_citation=FileCitation(file_id='file-25EnyP9GxDFbVhTMq8T2EM'), start_index=874, text='【4:0†source】', type='file_citation'), FileCitationAnnotation(end_index=898, file_citation=FileCitation(file_id='file-25EnyP9GxDFbVhTMq8T2EM'), start_index=886, text='【4:1†source】', type='file_citation'), FileCitationAnnotation(end_index=1099, file_citation=FileCitation(file_id='file-25EnyP9GxDFbVhTMq8T2EM'), start_index=1087, text='【4:1†source】', type='file_citation'), FileCitationAnnotation(end_index=1112, file_citation=FileCitation(file_id='file-25EnyP9GxDFbVhTMq8T2EM'), start_index=1099, text='【

In [8]:
id = thread_messages.data[0].id

In [9]:
# Retrieve the message object
message = client.beta.threads.messages.retrieve(
  thread_id=run.thread_id,
  message_id=id
)


In [10]:
print(message.content[0].text)

Text(annotations=[FileCitationAnnotation(end_index=559, file_citation=FileCitation(file_id='file-F8ec6VTMZo9WxFuSj5LUh4'), start_index=547, text='【4:2†source】', type='file_citation'), FileCitationAnnotation(end_index=886, file_citation=FileCitation(file_id='file-25EnyP9GxDFbVhTMq8T2EM'), start_index=874, text='【4:0†source】', type='file_citation'), FileCitationAnnotation(end_index=898, file_citation=FileCitation(file_id='file-25EnyP9GxDFbVhTMq8T2EM'), start_index=886, text='【4:1†source】', type='file_citation'), FileCitationAnnotation(end_index=1099, file_citation=FileCitation(file_id='file-25EnyP9GxDFbVhTMq8T2EM'), start_index=1087, text='【4:1†source】', type='file_citation'), FileCitationAnnotation(end_index=1112, file_citation=FileCitation(file_id='file-25EnyP9GxDFbVhTMq8T2EM'), start_index=1099, text='【4:19†source】', type='file_citation'), FileCitationAnnotation(end_index=1397, file_citation=FileCitation(file_id='file-GynE98zpGD5UMVNsU7pf7g'), start_index=1385, text='【4:3†source】', ty

In [11]:
def get_name_by_file_id(df_merge,file_id):
    result = df_merge[df_merge["file_id"]==file_id]
    citation_url = result['url'].values[0]
    return citation_url

from urllib.parse import urlparse

def is_valid_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

# Example usage:
test_url = "https://www.example.com"
print(is_valid_url(test_url))  # Output: True

invalid_url = "not_a_url"
print(is_valid_url(invalid_url))  # Output: False

True
False


In [12]:
# Extract the message content
message_content = message.content[0].text.model_copy()
print(message_content.value)

You're looking for ways to learn about meditation, which is a fantastic pursuit for enhancing your well-being! Here are several resources and opportunities available at the University of Oregon that can help you get started:

1. **Duck Nest Wellness Center**: The Duck Nest offers **Meditation Mondays**, which are student-led meditation sessions. These workshops provide a welcoming environment to explore various meditation practices. Sessions are held every Tuesday from 4:00 PM to 4:30 PM at EMU 041. You can drop in without prior registration【4:2†source】.

2. **Mindfulness Video Playlist**: University Counseling Services has created a Mindfulness Video Playlist that includes instructional videos on different mindfulness practices. Topics covered include "Introduction to Mindfulness," "Grounding in the Breath," "Grounding in the Body," and "Grounding in Gratitude"【4:0†source】【4:1†source】.

3. **WellTrack App**: This mobile application features a component called "The Zen Room," which is 

In [13]:
# Extract the message content
message_content = message.content[0].text.model_copy()
annotations = message_content.annotations
citations = pd.DataFrame()
cite_idx = 1

# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):

    
    if annotation.file_citation.file_id not in [row['file_id'] for _, row in citations.iterrows()]:

       this_citation = pd.DataFrame({'original': annotation.text, 'numeric': cite_idx, 'file_id': annotation.file_citation.file_id}, index = [0]) #Need to make these Pandas Dataframes!!
       citations = pd.concat([citations,this_citation],axis=0, ignore_index=True)
       cite_idx = cite_idx + 1

    else: 
      prev_indices = citations.index[citations['file_id'] == annotation.file_citation.file_id].to_list()
      prev_entry = citations.iloc[prev_indices[0]]
      same_cite = pd.DataFrame({'original': annotation.text, 'numeric': prev_entry['numeric'], 'file_id': annotation.file_citation.file_id},index=[0])
      citations = pd.concat([citations,same_cite],axis=0, ignore_index=True)
       

for index, citation in citations.iterrows():

    original = citation['original']
    numeric = citation['numeric']

    this_url =  get_name_by_file_id(df_merge,citation['file_id'])

    # Replace the text with a footnote
    message_content.value = message_content.value.replace(original, f' [Source {numeric}]({this_url})')





    
    



In [14]:
print(message_content.value)

You're looking for ways to learn about meditation, which is a fantastic pursuit for enhancing your well-being! Here are several resources and opportunities available at the University of Oregon that can help you get started:

1. **Duck Nest Wellness Center**: The Duck Nest offers **Meditation Mondays**, which are student-led meditation sessions. These workshops provide a welcoming environment to explore various meditation practices. Sessions are held every Tuesday from 4:00 PM to 4:30 PM at EMU 041. You can drop in without prior registration [Source 1](https://counseling.uoregon.edu/mham).

2. **Mindfulness Video Playlist**: University Counseling Services has created a Mindfulness Video Playlist that includes instructional videos on different mindfulness practices. Topics covered include "Introduction to Mindfulness," "Grounding in the Breath," "Grounding in the Body," and "Grounding in Gratitude" [Source 2](https://counseling.uoregon.edu/copingstrategies) [Source 2](https://counseling

In [31]:
import re

def remove_adjacent_duplicate_links(markdown_string):
    # Define a regex pattern for markdown links
    markdown_link_pattern = r'\[.*?\]\(.*?\)'

    # Find all markdown links in the string
    all_links = re.findall(markdown_link_pattern, markdown_string)

    # Initialize an empty list to hold the processed links
    processed_links = []

    previous_link = None

    for link in all_links:
        if link != previous_link:
            processed_links.append(link)
        previous_link = link

    # Reconstruct the string with the processed links
    # Split the original text by the markdown pattern
    parts = re.split(markdown_link_pattern, markdown_string)

    # Interleave the text parts with the processed links
    result = parts[0]
    for link, part in zip(processed_links, parts[1:]):
        result += link + part

    return result

# Simulate some markdown input


# Remove adjacent duplicate links
cleaned_markdown_string = remove_adjacent_duplicate_links(message_content.value)

print("Processed markdown string without adjacent duplicate links:")
print(cleaned_markdown_string)

Processed markdown string without adjacent duplicate links:
You're interested in ways to learn about meditation! There are several valuable resources and opportunities available for you at the University of Oregon that can help you explore and develop your meditation practice.

1. **Mindfulness Resources**: The University Counseling Services has a Mindfulness Video Playlist that includes instructional videos on various mindfulness techniques, such as grounding in the breath and body, and mindfulness in daily activities [Source 1](https://counseling.uoregon.edu/copingstrategies).

2. **Apps and Online Resources**: You can explore popular meditation and mindfulness apps like Headspace, Calm, and WellTrack, which includes a tool called the Zen Room to help you clear your mind and relax [Source 2](https://counseling.uoregon.edu/mham).

3. **Meditation Workshops**: 
   - The Duck Nest Wellness Center offers **Meditation Mondays**, which are student-led meditation sessions that take place in

In [30]:
print(message_content.value)

You're interested in ways to learn about meditation! There are several valuable resources and opportunities available for you at the University of Oregon that can help you explore and develop your meditation practice.

1. **Mindfulness Resources**: The University Counseling Services has a Mindfulness Video Playlist that includes instructional videos on various mindfulness techniques, such as grounding in the breath and body, and mindfulness in daily activities [Source 1](https://counseling.uoregon.edu/copingstrategies).

2. **Apps and Online Resources**: You can explore popular meditation and mindfulness apps like Headspace, Calm, and WellTrack, which includes a tool called the Zen Room to help you clear your mind and relax [Source 1](https://counseling.uoregon.edu/copingstrategies).

3. **Meditation Workshops**: 
   - The Duck Nest Wellness Center offers **Meditation Mondays**, which are student-led meditation sessions that take place in the Erb Memorial Union (EMU). This can be a gre

In [ ]:
#Upload txt files 

#Upload 

In [16]:

# Extract the message content
message_content = message.content[0].text
annotations = message_content.annotations
citations = []

# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):
    # Replace the text with a footnote
    message_content.value = message_content.value.replace(annotation.text, f' [{index+1}]')
    
    #Get the File ID for the file in this citation
    file_id = annotation.file_citation.file_id

    #Find the URL or source for the file_id that OpenAI has
    citation = get_name_by_file_id(df_merge,file_id)

    # Add the citation to the list
    citations.append(f'[{index+1}] {citation}')

# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n' + 'Sources' + '\n' + '\n'.join(citations)

In [17]:
print(message_content.value)

You asked about Lauren Eichler. She is the Assistant Director for Advisor Education and Training in the Office of Academic Advising at the University of Oregon.

Lauren Eichler has an academic background that includes a PhD in Philosophy from the University of Oregon. Her research interests span several fields, including environmental ethics, genocide studies, Native American philosophies, and ethics in general. Before her current role, she taught philosophy at Old Dominion University, where she also served as the Director of Assessment for the Department of Philosophy and Religious Studies.

Eichler is deeply committed to academic advising and mentoring students, emphasizing the importance of liberal arts education and its impact on individuals and society. Her personal experiences and fondness for the UO community led her to return to the university, where she aims to make a meaningful contribution to student development [1].
[1] https://advising.uoregon.edu/-lauren-eichler
Sources
[

In [148]:
import time

formatted_string = message_content.value
delay = 0.01  # delay in seconds

for char in formatted_string:
    print(char, end='', flush=True)
    time.sleep(delay)

You're interested in learning about Lauren Eichler. She is currently the Assistant Director for Advisor Education and Training in the Office of Academic Advising at the University of Oregon.

Lauren Eichler has an extensive academic background, having grown up in Baltimore, MD. She earned her PhD in Philosophy from the University of Oregon after spending a decade in higher education and research. Her areas of interest in philosophy include environmental ethics, genocide studies, and Native American philosophies. Before returning to UO, she taught philosophy at Old Dominion University and served as the Director of Assessment in their Department of Philosophy and Religious Studies. 

In her role at UO, Lauren is deeply committed to mentoring students and guiding them in their academic and professional journeys. She describes advising as a way to engage in meaningful conversations that support students both practically and existentially [1] [2].

If you'd like to know more about her work 